In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/zillow-prize-1//train_2017.csv', parse_dates=["transactiondate"])
prop_df = pd.read_csv('../input/zillow-prize-1/properties_2017.csv',low_memory=False)

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

# import warnings
# warnings.filterwarnings("ignore")


plt.rcParams['figure.figsize'] = (8.5, 5)
plt.rcParams["patch.force_edgecolor"] = True
sns.mpl.rc("figure", figsize=(8.5,5))
pd.set_option('max_colwidth', 400)

# ANALYSE

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
prop_df.head()

In [ ]:
prop_df.shape

In [ ]:
#je fusionne les 2 jeux de données en un seul
df= train_df.merge(prop_df, how='left', on='parcelid')

In [ ]:
df.shape

# CLEANING

##  Logerror:

Observation de la variable cible

In [ ]:
plt.scatter(range(df.shape[0]), np.sort(df.logerror.values))
plt.xlabel('index', fontsize=12)
plt.ylabel('logerror', fontsize=12)
plt.show()

## Extreme values

In [ ]:
# choose the max and min value 
logerror_max = np.percentile(df.logerror.values, 99)
logerror_min = np.percentile(df.logerror.values, 1)

df = df[df['logerror']<logerror_max]
df = df[df['logerror']>logerror_min]

sns.distplot(df.logerror.values, bins=50, kde=False)
plt.xlabel('logerror', fontsize=12)
plt.show()

## Duplicates

In [ ]:
print("Nombre de lignes dupliquées", df.duplicated().sum())

## Missing values

In [ ]:
print("Nombre de valeurs manquantes", df.isna().sum().sum())

In [ ]:
# delete columns with missing values > 40%
df = df.loc[:, df.isna().sum() < 0.4*df.shape[0]]

In [ ]:
# keep only numeric variabls
df_quant = df.select_dtypes(include=['float64','int64'])

In [ ]:
df_nan = df_quant.isna().sum().sort_values()


### Correlation


In [ ]:
corr = df_quant.corr()
corr10 = corr['logerror'].abs().sort_values(ascending=False)
corr10.head(11)

Aucune des variables ne presente de correlation forte, ici le model de regression lineaire n'a pas de sens.

# MACHINE LEARNING

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error 
from sklearn.tree import DecisionTreeRegressor

import xgboost as xgb

In [ ]:
def get_metrics(data, target, model):
    ''' 
    Get R2, RMSE and MAE for a model
    data = X
    target = y
    '''
    X_train, X_test, y_train, y_test = train_test_split(data, target, train_size=0.8, random_state=1)
    model = model
    model.fit(X_train, y_train)
    ytrainpred = model.predict(X_train)
    ytestpred = model.predict(X_test)
    print(model)
    print ("Training set : R2 = {} RMSE = {}\nTesting set : R2 = {} RMSE = {}" 
      .format(round(r2_score(y_train, ytrainpred),3), round(mean_squared_error(y_train, ytrainpred, squared=False),3), round(r2_score(y_test, ytestpred),3), round(mean_squared_error(y_test, ytestpred, squared=False),3)))
    print("Training Set : Mean Absolute Error = {}".format(mean_absolute_error(ytrainpred, y_train)))
    print("Testing Set : Mean Absolute Error = {}".format(mean_absolute_error(ytestpred, y_test)))

In [ ]:
# definit target
y = df.logerror

# defniti explanation's variables
df_quant = df_quant.drop(columns = ['logerror'])
X = df_quant

In [ ]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X = pd.DataFrame(my_imputer.fit_transform(X))

# Imputation removed column names; put them back
imputed_X.columns = X.columns

X = imputed_X.copy()

## REGRESSION LINEAIRE

meme si nous avons vu plus haut qu'il n'existait pas de correlation lineaire antre notre target et une variable quantitative, je vais vérifier la possible existence d'une telle correlation avec les variables categorielles.

- propertycountylandusecode:  County land use code i.e. it's zoning at the county level

- propertyzoningdesc:  Description of the allowed land uses (zoning) for that property

In [ ]:
df_cat = df[['propertycountylandusecode','propertyzoningdesc']]

In [ ]:
new_df_cat = pd.get_dummies(df_cat, prefix=['pcluc', 'pzd'])

In [ ]:
new_df_cat.shape

In [ ]:
new_df_cat.head()

## Regression lineaire multiple avec regulation Ridge

In [ ]:
from sklearn import linear_model
regr = linear_model.Ridge(alpha=100)
get_metrics(new_df_cat, y, regr)

ces variables categorielle n'ont pas assez de correlation avec logerror pour etre pertinentes

## DecisionTreeRegression 

j'utilise ce model pour definir quelles sont les variables les plus utiles à la prediction.

In [ ]:
get_metrics(X, y, DecisionTreeRegressor(random_state=1))

## RANDOMFOREST

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)
model = RandomForestRegressor(random_state=1)
model.fit(X_train, y_train)
ytrainpred = model.predict(X_train)
ytestpred = model.predict(X_test)
print(model)
print ("Training set : R2 = {} RMSE = {}\nTesting set : R2 = {} RMSE = {}" .format(round(r2_score(y_train, ytrainpred),3), round(mean_squared_error(y_train, ytrainpred, squared=False),3), round(r2_score(y_test, ytestpred),3), round(mean_squared_error(y_test, ytestpred, squared=False),3)))
print("Training Set : Mean Absolute Error = {}".format(mean_absolute_error(ytrainpred, y_train)))
print("Testing Set : Mean Absolute Error = {}".format(mean_absolute_error(ytestpred, y_test)))

In [ ]:
a = model.feature_importances_
np.sort(a)  

In [ ]:
name = X.columns
corr = model.feature_importances_
df = pd.DataFrame({'name':name, 'corr':corr})
df.sort_values(by='corr', ascending = False)
# df.plot('name', 'corr', kind='scatter')

## XGBOOST

In [ ]:

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.15)

xgbr = xgb.XGBRegressor(verbosity=0)
print(xgbr)

xgbr.fit(xtrain, ytrain) 

score = xgbr.score(xtrain, ytrain)   

print("Training score: ", score)  

# - cross validataion 
scores = cross_val_score(xgbr, xtrain, ytrain, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbr, xtrain, ytrain, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean()) 

ypred = xgbr.predict(xtest)
mse = mean_squared_error(ytest, ypred)
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))


x_ax = range(len(ytest))
plt.scatter(x_ax, ytest, s=5, color="blue", label="original")
plt.plot(x_ax, ypred, lw=0.8, color="red", label="predicted")
plt.legend()
plt.show()

In [ ]:
print("Training Set : Mean Absolute Error = {}".format(mean_absolute_error(ytrainpred, y_train)))
print("Testing Set : Mean Absolute Error = {}".format(mean_absolute_error(ytestpred, y_test)))

# SUBMISSION

In [ ]:
# read the sample_submisison file into a dataframe, for sake of clarity let's limit the number of rows to 5
df_samples = pd.read_csv('../input/zillow-prize-1/sample_submission.csv')

In [ ]:
# let's unpivot the dataframe to produce the shape regression models usually expect
# consisting of pairs `parcelid`, `transactiondate``

def reshape_for_model(df):
    """Unipivot the submission data and apply some renamings"""
    df = pd.melt(df, ['ParcelId'])
    df.drop('value', axis=1, inplace=True)
    df.columns = ['parcelid', 'transactiondate']
    df['transactiondate'] = df['transactiondate'].apply(
        lambda date_str: "%s-%s-01" %(date_str[:4], date_str[-2:]))
    return df

In [ ]:
df_reshaped = reshape_for_model(df_samples)

In [ ]:
X.columns
new_df = prop_df[['parcelid', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'fips', 'fullbathcnt', 'heatingorsystemtypeid',
       'latitude', 'longitude', 'lotsizesquarefeet', 'propertylandusetypeid',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'unitcnt', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock']]


# new_df = new_df.sample(n=10000)


In [ ]:

p_1 = xgbr.predict(new_df)
p_2 = xgbr.predict(new_df)
p_3 = xgbr.predict(new_df)
p_4 = xgbr.predict(new_df)
p_5 = xgbr.predict(new_df)
p_6 = xgbr.predict(new_df)


In [ ]:
pred = np.concatenate((p_1, p_2, p_3, p_4, p_5, p_6), axis=None)

In [ ]:
df_reshaped['prediction']  = pred

In [ ]:
###### to get the original shape of our submission sample file, we can now pivot the table again
# and remove the column names
def reshape_for_submission(df):
    """Reformat the transactiondate and pivot the data"""
    df['transactiondate'] = df['transactiondate'].apply(lambda td: "%s%s" %(td[:4],td[5:7]))
    df = df.pivot(index='parcelid', columns='transactiondate', values='prediction')
    df = df.reset_index()
#     df = df.rename_axis(None,1)
    df = df.rename(index=str, columns={"parcelid": "ParcelId"})
    return df

In [ ]:
df_tosubmit = reshape_for_submission(df_reshaped)

In [ ]:
df_tosubmit.head()

In [ ]:
df_tosubmit.to_csv('final.csv',index = False)